## Create a Chroma Client

In [1]:
import chromadb
import gradio as gr
import os

/home/bigdata/miniconda3/envs/chroma/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Creación da cliente Chroma

In [2]:
chroma_client = chromadb.Client()

### 2. Creación da colección

In [3]:
collection = chroma_client.create_collection(name="documents")

### 3. Creación da función que distingue se é un JSON e permiteo subir gracias a Gradio

In [4]:
def subir_archivos(files):
    
    if not files:
        return "No se seleccionaron archivos."

    docs = []
    ids = []
    metadatas = []

    try:
        for f in files:
            # f.name es la ruta al archivo temporal subido por Gradio.
            filepath = f.name
            
            # Obtener el nombre de archivo original para usarlo como ID y en metadatos.
            # Usamos os.path.basename para asegurarnos de que solo tenemos el nombre del archivo.
            filename = os.path.basename(filepath) 
            
            # 💡 Corrección clave: Abrir y leer el archivo usando su ruta temporal.
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()

            docs.append(content)
            ids.append(filename)
            metadatas.append({"filename": filename})

        # Añadir documentos a la colección
        collection.add(
            documents=docs,
            ids=ids,
            metadatas=metadatas
        )

        return f"{len(files)} archivo(s) subido(s) correctamente a la colección 'documents'."

    except Exception as e:
        return f"Error al subir archivos: {str(e)}"


### 4. Creación da función que permite realizar as consultas e que devolve o ficheiro que determina que máis se aproxima

In [5]:
def consultar_documento(query):
    
    try:
        # ChromaDB usa su modelo de embeddings por defecto para la consulta semántica
        result = collection.query(
            query_texts=[query],
            n_results=1
        )

        
        # Extraer el documento y metadatos del resultado (el primer resultado [0] del primer query [0])
        documento = result["documents"][0][0]
        filename = result["metadatas"][0][0]["filename"]
        
        # Opcional: Obtener la distancia/score si está disponible
        score = result["distances"][0][0]

        return (
            f"Documento más relevante encontrado:\n"
            f"--- Nombre de Fichero: **{filename}** ---\n"
            f"--- Relevancia (Distancia): **{score:.4f}** ---\n\n"
            f"Contenido:\n{documento}"
        )

    except Exception as e:
        return f"Error en la consulta: {str(e)}"


### 5. Creación da interface gráfica que permite subir os archivos e realizar as consultas mediante Gradio

In [ ]:
with gr.Blocks(title="Buscador Semántico de Documentos (ChromaDB)") as demo:
    gr.Markdown("# 📚 Buscador Semántico con ChromaDB (In-Memory)")

    with gr.Tab("📂 Subir Archivos"):
        gr.Markdown("### Sube tus documentos de texto (.json) aquí.")
        # Se restringe a .json según los requisitos opcionales/prueba
        file_input = gr.File(
            file_types=[".json"], 
            file_count="multiple", 
            label="Selecciona Ficheros  JSON"
        )
        subir_btn = gr.Button("🚀 Subir Documentos")
        status = gr.Textbox(label="Estado de la Operación", lines=2)
        
        subir_btn.click(
            fn=subir_archivos, 
            inputs=file_input, 
            outputs=status
        )
    
    with gr.Tab("🔍 Consulta Semántica"):
        gr.Markdown("### Introduce tu pregunta para encontrar el documento más relevante.")
        query_input = gr.Textbox(label="Introduce tu pregunta o consulta")
        consultar_btn = gr.Button("🔎 Consultar")
        resultado = gr.Textbox(label="Resultado de la Búsqueda", lines=10)
        
        consultar_btn.click(
            fn=consultar_documento, 
            inputs=query_input, 
            outputs=resultado
        )

# Iniciar la interfaz
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
